In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pytesseract
import re
from playsound import playsound

img = cv2.imread('./IMG_0000.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img2 = img
# plt.imshow(RGB_img)


# threshold on yellow color
lower=(140,30,30)
upper=(170,60,60)
mask = cv2.inRange(img, lower, upper)

# change all non-yellow to white
filtered = img.copy()
filtered[mask!=255] = (255, 255, 255)



img1 = cv2.cvtColor(filtered, cv2.COLOR_BGR2GRAY)
# plt.imshow(img1)

ret,thresh = cv2.threshold(img1,150,255,0)
contours,hierarchy = cv2.findContours(thresh, 1, 2)
print("Number of contours detected:",len(contours))
min_y = 999999
string = None
arrow_x = None
arrow_y = None

for i in range(len(contours)):
    if hierarchy[0][i][3] == -1 and hierarchy[0][i][1] > 10  :
        cv2.drawContours(img, contours, i, (0,255,255), 3)
    approx = cv2.approxPolyDP(contours[i], 0.009 * cv2.arcLength(contours[i], True), True) 
    n = approx.ravel()
    i2 = 0
    
    for j in n : 
        if(i2 % 2 == 0): 
            x = n[i2] 
            y = n[i2 + 1] 

            # String containing the co-ordinates. 
            if (min_y > y.item() and y.item() > 10):
                string = str(x) + " " + str(y) 
                min_y = y.item()
                arrow_x = x.item()
                arrow_y = y.item()
        i2 = i2 + 1
cv2.putText(img, "Arrow tip", (arrow_x, arrow_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))


# plt.imshow(img, cmap='gray')

# save results
cv2.imwrite('contours.jpg',img)


In [ ]:
# Import required packages


print(arrow_x)
print(arrow_y)

# Mention the installed location of Tesseract-OCR in your system
# pytesseract.pytesseract.tesseract_cmd = '/usr/local/Cellar/tesseract/5.3.1/bin/tesseract.exe'

# Read image from which text needs to be extracted

img2 = img2[(arrow_y - 100) :arrow_y + 10 ,(arrow_x - 100) : (arrow_x + 9999)] 
# plt.imshow(img, cmap='gray')

# Preprocessing the image starts

# Convert the image to gray scale
gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Performing OTSU threshold
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

# Specify structure shape and kernel size.
# Kernel size increases or decreases the area
# of the rectangle to be detected.
# A smaller value like (10, 10) will detect
# each word instead of a sentence.
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (19,19 ))

# Applying dilation on the threshold image
dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

# Finding contours
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
												cv2.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = img2.copy()


# A text file is created and flushed
# file = open("recognized.txt", "w+")
# file.write("")
# file.close()

# Looping through the identified contours
# Then rectangular part is cropped and passed on
# to pytesseract for extracting text from it
# Extracted text is then written into the text file
for cnt in contours:
	x, y, w, h = cv2.boundingRect(cnt)
    
    
# 	print(str(diff_x) + " " + str(diff_y)) 
	# Drawing a rectangle on copied image
	rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)

	# Cropping the text block for giving input to OCR
	cropped = im2[y:y + h, x:x + w]

	# Apply OCR on the cropped image
	text = pytesseract.image_to_string(cropped, config='digits')

	# Appending the text into file
	if re.sub('\D', '', text) != '':
		print("hit" + str(x) + " " + str(y) + " " + re.sub('\D', '', text)) 
		playsound( re.sub('\D', '', text) + ".wav")
plt.imshow(im2, cmap='gray')
